# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    try:
    # reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)

     # extracting each data row one by one and append it        
            for line in csvreader:
                full_data_rows_list.append(line) 
    except Exception as e:
        print(e)
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


2823


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

2411


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Task 1: Get Artist, Song Title and Song Length by Session ID and Item in Session

#### Create the session_history table
This table enables sessionId, iteminSession queries for artist, song_title and song_length. The PRIMARY KEY is composite, partitioning on sessionId, clustering on itemInSession because the sessionId
should be reasonable evenly distributed and we need to be able to extract items clustered by itemInSession,

In [10]:
query = "CREATE TABLE IF NOT EXISTS session_history "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### INSERT Session History

In [11]:
# Insert CSV data into the data table session_history
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], round(float(line[5]))))

#### SELECT Query -> Artist, Song Title and Song Length

In [12]:
# Select all listen data (sessionId, artist, song_title, song_length) from a particular session
query = "select artist, song_title, song_length from session_history WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length,)

# Task 2: Get Artist, Song and User by User ID and Session ID

#### Create the session_songs table
This table enables sessionId, userId, itemInSession queries for artist, song queries for a given user (first and last name) and orders by their play order in a session. The PRIMARY KEY is composite, partitioning on userId and sessionId, clustering on itemInSession. This spreads user sessions across clusters and allows you to cluster and order by itemInSession.

In [13]:
query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song_title text,first_name text, last_name text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### INSERT Session Songs

In [14]:
# Insert CSV data into the data table session_songs
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT Query -> Artist, Song Title, First Name, Last Name

In [18]:
# Select all songs from the a particular session for a given user ID
query = "select artist, song_title, first_name, last_name from session_songs WHERE sessionId=182 AND userId=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name,)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Task 3: Get Users Who've Listened to a Particular Song

In [19]:
query = "CREATE TABLE IF NOT EXISTS song_listened "
query = query + "(song_title text, first_name text, last_name text, userId int, PRIMARY KEY (song_title, first_name, last_name, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                    

### INSERT Song Listened

In [21]:
# Insert CSV data into the data table song_listened
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listened (song_title, first_name, last_name, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

### SELECT Query -> First Name, Last Name

In [23]:
# Select all users who've listened to the song 'All Hands Against His Own'
query = "select first_name, last_name from song_listened WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name,)

Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [24]:
query = "drop table session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_listened"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()